## О данных

Данные взяты из всех статей сайта ixbt.com с 2020 года до текущего времени (май 2024). Большая часть статей -- обзоры на компьютерную технику, в которых содержится очень много английских слов, терминов и технической информаци. 

Вот как вяглядит отрывок из типичной статьи:

*Весной этого года компания HP представила две новые модели ноутбуков на мобильных процессорах AMD Ryzen 4000: ProBook 445 G7 с дисплеем 14 дюймов и ProBook 455 G7 с 15,6-дюймовым дисплеем. Обе версии выпущены в алюминиевом корпусе, могут оснащаться процессорами Ryzen 3 4300U, Ryzen 5 4500U или Ryzen 7 4700U, иметь до 32 ГБ оперативной памяти стандарта DDR4-3200 и SSD-накопитель объемом до 512 ГБ. Особое внимание в HP уделили обеспечению безопасности ноутбуков, реализованной в данных моделях на аппаратном и программном уровнях. Но все же главное в новых ProBook G7, на наш взгляд, это сочетание производительности и автономности при сравнительно компактном корпусе.Сегодня мы познакомимся с «большой» моделью HP ProBook 455 G7, конфигурация которой далека от максимальной, что позволяет ноутбуку выгодно вписаться в средний ценовой диапазон.*

Как можно видеть, здесь содержится много цифр, кодов и аббревиатур, что усложняет анализ текста. 

В корпусе есть и более простые тексты (допустим, обзор блинницы и описание процесса приготовления), но таких явно меньшинство.

## Качество данных.

Данные собирались простым скрапингом html-страниц. Отсюда появились некоторые артефакты в данных: 
1) При склейке параграфов статьи в один текст, пропадал пробел между ними. 
   Вот пример: *Корпус и внутреннюю поверхность крышки можно протереть влажной тканью.Будучи просто включенной в розетку, PMC 0593AD потребляет 0,3 Вт.За 15 минут в режиме «Крупа» энергопотребление составило 0,15 кВт·ч.*

   Данный артефакт плохо обрабатывается разными библиотеками, и 3 предложения выше будет распознаны как одно. 

2) В текстах попадаются другие артефакты. Один из самых явных (и легко исправимых), это приписка `Плюсы:Минусы:`. Иногда в середине предложения
есть какие-то секции, которых как будто там не должно быть (подписи к нетекстовым элементам стрницы, таким как таблицы или картинки). Но у таких дефектов я не нашел какой-то общей структуры, поэтому не особо понимаю, как их можно удалить.

3) Часто встречаются "склеенные" слова на стыке предложений. Поэтому решил разбить комбинации <буква в нижнем регистре><буква в верхнем регистре> в <буква в нижнем регистре>'.'<буква в верхнем регистре>. 
Неправильных разбиений будет сильно меньше, чем сейчас в текстах таких проблем.


## Инструмены

В качестве основной библиотеки решил использовать библиотеку spaCy. И модель `ru_core_news_sm`. Точность можно посмотреть по [ссылке](https://spacy.io/models/ru#ru_core_news_sm).

Пробовал использовать библиотеку `nltk`. На субъективный взгляд, она хуже справляется с текстами такого характера, где много технических терминов, наименований и аббревиатур.
Также у нее более скудные возможности морфологического анализа.

Пробовал и stanza. Она выдает сильно лучшее качество NER, но в остальном примерно так же. Проблема в том, что она раз в 20 медленнее spaCy, и на обработку всех данных
ушло бы около 5 часов.

Для поиска лексических омонимов будет использоваться BERT, так как он дает возможность получать контекстные эмбеддинги.

## Обработка данных

In [29]:
import os
import re
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from spacy import displacy

In [30]:
def fix_data(text):
    text = re.sub(r'(?<=[.])(?=[А-ЯЁ])', r' ', text)
    text = re.sub('Плюсы:Минусы:', '', text)
    text = re.sub(r'(?<=[а-яё])(?=[А-ЯЁ])', r'. ', text)
    return text

In [31]:
def split_text(text, length):
    poss = []
    for start in range(length, len(text), length):
        for i in range(start, start-length, -1):
            if text[i] == '.' and text[i+1] == ' ':
                poss.append(i)
                break
    chunks = []
    start = 0
    for pos in poss:
        chunks.append(text[start:pos+1])
        start = pos+2
    chunks.append(text[start:])
    return chunks

In [32]:
merged_texts_chunks = []
for file in os.walk("articles"):
    merged_texts = ""
    cnt = 0
    for filename in file[2]:
        cnt += 1
        with open(f"articles/{filename}", "r") as f:
            merged_texts += f.read()
            if len(merged_texts) > 1000000:
                if len(merged_texts) > 2000000:
                    chunks = split_text(merged_texts, 1000000)
                    for chunk in chunks:
                        merged_texts_chunks.append(chunk)
                else:
                    merged_texts_chunks.append(merged_texts)
                cnt = 0
                merged_texts = ""

if cnt != 0:
    merged_texts_chunks.append(merged_texts)

print(len(merged_texts_chunks))

27


In [33]:
print([len(chunk) for chunk in merged_texts_chunks])

[1002053, 1014614, 1006778, 1009496, 1013003, 1385272, 999770, 999916, 1000249, 999815, 1000015, 999830, 505185, 1005357, 1022153, 1007434, 1023565, 1022493, 1256923, 1305926, 1013223, 999703, 999741, 101706, 1001733, 1001219, 938045]


In [34]:
for i, chunk in enumerate(tqdm(merged_texts_chunks)):
    merged_texts_chunks[i] = fix_data(chunk)

100%|██████████| 27/27 [00:00<00:00, 39.52it/s]


In [ ]:
nlp = spacy.load("ru_core_news_md")
nlp.add_pipe("sentencizer")
nlp.max_length = 2e6

docs = []
for text in tqdm(merged_texts_chunks[:1]):
    doc = nlp(text)
    docs.append(doc)

doc = Doc.from_docs(docs)

In [9]:
doc.to_disk("articles_clean_vectors.spacy")

## Предварительные результаты

### Статистика

In [19]:
nlp = spacy.load("ru_core_news_md")
doc = Doc(nlp.vocab).from_disk("articles_clean1.spacy")

In [20]:
print('Всего предложений:', len(list(doc.sents)))
print('Всего токенов:', len(doc))
print('Non stop words:', len([token for token in doc if not token.is_stop]))

Всего предложений: 203444
Всего токенов: 4524841
Non stop words: 3205591


In [21]:
from collections import Counter
words = [token.text for token in doc if token.is_alpha and not token.is_stop]
word_freq = Counter(words)
print(*(word_freq.most_common(20)), sep='\n')

('USB', 5994)
('случае', 5771)
('работы', 5263)
('камеры', 5034)
('время', 4855)
('режиме', 4734)
('выше', 4635)
('экрана', 4449)
('C', 4341)
('питания', 4311)
('помощью', 4168)
('ГБ', 4158)
('RTX', 4157)
('температуры', 3900)
('яркости', 3821)
('меню', 3801)
('памяти', 3793)
('устройства', 3700)
('PCIe', 3391)
('два', 3336)


Выше приведен список из самых часто встречающихся слов в тексте за исключением самых часто употребляемых в языке (стоп-слов). В целом, из этого списка можно даже сделать вывод
о тематике корпуса.

Ниже тот же список, но для инфинитивов.

In [22]:
words = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

word_freq = Counter(words)
print(*(word_freq.most_common(20)), sep='\n')

('режим', 12374)
('камера', 12178)
('устройство', 10816)
('экран', 9833)
('работа', 9331)
('настройка', 9006)
('время', 8980)
('температура', 8517)
('два', 7679)
('случай', 7530)
('возможность', 6690)
('яркость', 6530)
('уровень', 6390)
('usb', 5994)
('программа', 5921)
('модель', 5903)
('система', 5797)
('тест', 5667)
('использовать', 5578)
('процессор', 5490)


### Качество

Сейчас глазами поищем интересные моменты в текстах стетей, а потом попытаемся эти моменты найти в проанализированных данных.

In [23]:
for sent in doc.sents:
    if 'Разогрели' in sent.text:
        print(sent.text)


Разогрели 2 ст. л.
Разогрели гриль с камнем до 210 градусов, положили на камень пиццу, выдержали 5 минут.
Разогрели гриль в ручном режиме.
Разогрели гриль в автоматическом режиме «Кебаб» до 210 °C, выложили на панель нанизанные на шпажки кусочки филе с помидорами и закрыли крышку.
Разогрели масло до 170 °C.
Разогрели мультиварку на программе «жарка», налили растительного масла, положили курицу и лук.


Стоит обратить внимание на первое предложение. На самом деле, оно было таким: *Разогрели 2 ст. л. масла.*

Но парсер не знает аббревиатуры `л.`, хотя правильно обработал `ст.` Таких примеров достаточно много. 

Затем еще выяснилась проблема. Если предложение заканчивается на аббревиатуру, то предложения склеиваются (очень часто такое происходило для Вт.).

Теперь посмотрим, как работает разбор относительно сложного предложения.

In [24]:
for sent in doc.sents:
    if 'хмели-сунели' in sent.text:
        print(sent.text)
        displacy.render(sent, style='dep')
        break

Наконец в чашу отправились помидоры, а чуть погодя и курица вернулась, но не одна, а с хмели-сунели.


Во-первых, "хмели-сунели" не распозналось, как единая синтаксическая единица. 
Во-вторых, сунели (и даже хмели-сунели) не является подлежащим.

Вот более успешный пример.

In [25]:
displacy.render(list(doc.sents)[0], style='dep')

Предложение все еще не элементарное, но разбор выполнен верно. Только немного лишних связей в названиях смартфона.

Теперь посмотрим, как работает NER.

In [26]:
for sent in list(doc.sents)[:10]:
    displacy.render(sent, style='ent')

In [27]:
print('Entities:', *((ent.text, ent.label_) for ent in doc.ents[:20]), sep='\n')

Entities:
('Realme', 'ORG')
('Индии', 'LOC')
('маркий', 'LOC')
('Xiaomi', 'ORG')
('Huawei', 'ORG')
('ИК-передатчик', 'ORG')
('Corning Gorilla Glass 5', 'ORG')
('Realme 6 Pro', 'ORG')
('Nexus', 'ORG')
('Гц', 'PER')
('IPS', 'ORG')
('микрофотографий экранов', 'PER')
('Nexus', 'ORG')
('экранов', 'PER')
('К. Перпендикулярно', 'PER')
('Яркие', 'LOC')
('одинаковая!):И', 'PER')
('Яркие', 'LOC')
('Яркие', 'LOC')
('Отметим', 'PER')


Честно, все плохо. В предложениях выше многие слова не распознаны, зато слово "маркий" оказалось локацией. В списке ниже тоже много слов, котоыре попасть были не должны. 

Еще странно, что "Realme 6 Pro" не распознался в первом предложении, хотя был распознан когда-то позднее.

In [28]:
sent = list(doc.sents)[0]
print(sent.text)
print()
for token in sent:
    print(token.text, token.morph)

Еще минувшей весной Realme представила в Индии смартфоны Realme 6 и Realme 6 Pro, называя их новыми флагманами.

Еще Degree=Pos
минувшей Case=Ins|Degree=Pos|Gender=Fem|Number=Sing
весной Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing
Realme Foreign=Yes
представила Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
в 
Индии Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
смартфоны Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur
Realme Foreign=Yes
6 
и 
Realme Foreign=Yes
6 
Pro Foreign=Yes
, 
называя Aspect=Imp|Tense=Pres|VerbForm=Conv|Voice=Act
их Animacy=Inan|Case=Acc|Number=Plur|Person=Third
новыми Case=Ins|Degree=Pos|Number=Plur
флагманами Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur
. 


Морфологический разбор строится верно в данном примере. И что приятно, отсюда можно вытянуть информацию о том, что слово иностранное. 

## Предварительные выводы
Качество разбиентия текста не предложения сносное. Вывел 200 предожений и нашел там 6 ошибок в разбиении (читал не очень внимательно). Проблемы известные: 
неверная обработка аббревиатур, склеенные предложения и наименования в конце предложения. 

Синтаксический разбор для несложных случаев работает неплохо. Но, как мы видели, есть примеры, где он сильно ошибается. 

Морфологический разбор, на первый взгляд, тоже строится верно. Проблемы стали видны при автоматичесом поиске омонимов.

NER у данной модели Spacy работает очень плохо. На самом деле, NER важен для текстов такого характера.

**P.S**. Пробовал взять другую модель для Spacy (бОльшую). Судя по описанию, там должны были появиться только векторные представления слов. Но на самом деле, стал чуть лучше синтаксический разбор первого предложения, и чуть лучше стал работать NER.

Еще пробовал NER у других библиотек. Например, у DeepPavlov он работает гораздо лучше. Но уже как-то не хочется загромождать ноутбук сравнением различных библиотек.